In [160]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC   
from sklearn.model_selection import GridSearchCV                 
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report

In [161]:
# test = pd.read_csv('testing.csv', header=None, skiprows=1)

train = pd.read_csv('training.csv')
validation = pd.read_csv('validation.csv', header=None, skiprows=1)
X = pd.read_csv('training.csv')


In [162]:
# replace labels with ints
train.replace('blues', 1, inplace=True)
train.replace('classical', 2, inplace=True)
train.replace('country', 3, inplace=True)
train.replace('disco', 4, inplace=True)
train.replace('hiphop', 5, inplace=True)
train.replace('jazz', 6, inplace=True)
train.replace('metal', 7, inplace=True)
train.replace('pop', 8, inplace=True)
train.replace('reggae', 9, inplace=True)
train.replace('rock', 10, inplace=True)
train.drop(['song_name'], axis=1, inplace=True)
train.head()

mean_spectral_rolloff  std_spectral_rolloff  mean_spectral_centroid  \
0            3805.839606            949.476395             1784.165850   
1            3042.260232            885.457204             1552.811865   
2            2184.745799           1221.963322             1070.106615   
3            3579.757627           1254.184130             1835.004266   
4            3481.517592           1809.541367             1831.993940   

   std_spectral_centroid  mean_spectral_bandwidth  std_spectral_bandwidth  \
0             360.241675              2002.449060              293.057608   
1             395.559911              1747.702312              276.141616   
2             429.366909              1596.412872              407.972419   
3             586.003361              1748.172116              297.397392   
4            1015.126775              1729.653287              449.344532   

   mean_spectral_flatness  std_spectral_flatness  mean_spectral_contrast  \
0                0.004498               0.005513               20.526699   
1                0.002631               0.003345               22.197517   
2                0.000954               0.001631               21.426268   
3                0.003238               0.004356               21.466338   
4                0.003061               0.006482               22.545831   

   std_spectral_contrast  ...  std_mfcc_15  mean_mfcc_16  std_mfcc_16  \
0               8.760242  ...     7.240229     -1.690215     6.043515   
1               8.725292  ...     6.371716     -7.729093     6.902132   
2               7.961446  ...     6.665415     -3.319596     7.085667   
3               7.936384  ...     9.278968     -5.454034     8.675812   
4               8.176806  ...     8.517583     -1.838263     8.288669   

   mean_mfcc_17  std_mfcc_17  mean_mfcc_18  std_mfcc_18  mean_mfcc_19  \
0     -0.408979     6.449582     -2.303523     7.420440      1.221290   
1     -1.816407     7.237551     -3.439720     6.829324     -2.231259   
2      0.636965     6.108939     -0.619121     6.104075     -3.407449   
3     -0.916874     7.322153     -4.404826     7.931634    -11.703234   
4     -2.783799     6.515171     -3.047909     6.309420     -8.109991   

   std_mfcc_19  label  
0     6.850987      1  
1     5.529287      1  
2     5.652375      1  
3     7.429345      1  
4     6.805219      1  

[5 rows x 77 columns]

In [163]:
# check to see if all labels are ints
train['label'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [164]:
X.drop(['song_name'], axis=1, inplace=True)
X.drop(['label'], axis=1, inplace=True)
X.head()

mean_spectral_rolloff  std_spectral_rolloff  mean_spectral_centroid  \
0            3805.839606            949.476395             1784.165850   
1            3042.260232            885.457204             1552.811865   
2            2184.745799           1221.963322             1070.106615   
3            3579.757627           1254.184130             1835.004266   
4            3481.517592           1809.541367             1831.993940   

   std_spectral_centroid  mean_spectral_bandwidth  std_spectral_bandwidth  \
0             360.241675              2002.449060              293.057608   
1             395.559911              1747.702312              276.141616   
2             429.366909              1596.412872              407.972419   
3             586.003361              1748.172116              297.397392   
4            1015.126775              1729.653287              449.344532   

   mean_spectral_flatness  std_spectral_flatness  mean_spectral_contrast  \
0                0.004498               0.005513               20.526699   
1                0.002631               0.003345               22.197517   
2                0.000954               0.001631               21.426268   
3                0.003238               0.004356               21.466338   
4                0.003061               0.006482               22.545831   

   std_spectral_contrast  ...  mean_mfcc_15  std_mfcc_15  mean_mfcc_16  \
0               8.760242  ...      0.752740     7.240229     -1.690215   
1               8.725292  ...      2.451690     6.371716     -7.729093   
2               7.961446  ...      0.780873     6.665415     -3.319596   
3               7.936384  ...     -4.520575     9.278968     -5.454034   
4               8.176806  ...     -5.576590     8.517583     -1.838263   

   std_mfcc_16  mean_mfcc_17  std_mfcc_17  mean_mfcc_18  std_mfcc_18  \
0     6.043515     -0.408979     6.449582     -2.303523     7.420440   
1     6.902132     -1.816407     7.237551     -3.439720     6.829324   
2     7.085667      0.636965     6.108939     -0.619121     6.104075   
3     8.675812     -0.916874     7.322153     -4.404826     7.931634   
4     8.288669     -2.783799     6.515171     -3.047909     6.309420   

   mean_mfcc_19  std_mfcc_19  
0      1.221290     6.850987  
1     -2.231259     5.529287  
2     -3.407449     5.652375  
3    -11.703234     7.429345  
4     -8.109991     6.805219  

[5 rows x 76 columns]

In [165]:
y = train['label'].copy()
y.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [166]:
# Normalization of Data
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X.iloc[:, :-1], dtype = float))


In [167]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [168]:
y_train.head()

575    8
108    2
46     1
132    2
220    3
Name: label, dtype: int64

In [169]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [170]:
# Quantile Transformer reduces accuracy
#Conversion from a col vector to a 1D array
#y_test=np.ravel(y_test) 
#y_train=np.ravel(y_train) 

#qt = QuantileTransformer(n_quantiles=50, random_state=0)         # This section normalize the data csv

#X_train=qt.fit_transform(X_train)                                 
#X_train=pd.DataFrame(X_train) 

#X_test=qt.fit_transform(X_test)                                 
#X_test=pd.DataFrame(X_test) 


In [171]:
#y_test.head()
y_test.shape

(186,)

In [172]:
# C: how much you want to avoid misclassifying each training example.
# # Gamma: how far the influence of a single training example reaches

parameters = {'kernel':('linear', 'rbf'),
              'C':[1, 10, 20]
             }

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

cls=SVC()
model = GridSearchCV(cls, param_grid)
model.fit(X_train,y_train,groups=None)


GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}])

In [173]:
# Testing 

test_pred = model.predict(X_test)

np.savetxt("accuracy_solution.csv", 
           np.dstack((np.arange(1, test_pred.size+1),test_pred))[0],
           delimiter=',', comments="", fmt='%i,%i',
           header="Sample_id,Sample_label")

In [174]:
print("Accuracy Score: ", accuracy_score(y_test, test_pred))

conf_matrix = confusion_matrix(y_test, test_pred)
print('\033[1m' + "\n Table 1: Confusion Matrix of Training Data \n\n" + '\033[0m', conf_matrix)

Accuracy Score:  0.7849462365591398

 Table 1: Confusion Matrix of Training Data 

 [[17  0  2  0  0  0  1  0  0  1]
 [ 0 17  1  0  0  1  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  2]
 [ 0  0  1 16  1  0  0  0  0  1]
 [ 0  0  0  2 12  0  0  0  3  0]
 [ 0  0  0  0  0 18  0  0  0  1]
 [ 0  0  0  0  0  0 17  0  0  2]
 [ 0  0  2  1  3  0  0 11  0  1]
 [ 1  0  1  2  1  0  0  0 13  0]
 [ 1  0  2  3  0  1  1  0  1  9]]


In [176]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           1       0.89      0.81      0.85        21
           2       1.00      0.89      0.94        19
           3       0.64      0.89      0.74        18
           4       0.67      0.84      0.74        19
           5       0.71      0.71      0.71        17
           6       0.90      0.95      0.92        19
           7       0.89      0.89      0.89        19
           8       1.00      0.61      0.76        18
           9       0.76      0.72      0.74        18
          10       0.53      0.50      0.51        18

    accuracy                           0.78       186
   macro avg       0.80      0.78      0.78       186
weighted avg       0.80      0.78      0.79       186

